In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime as dt
import random
from sportsreference.ncaab.teams import Teams
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
teams_df = pd.DataFrame()
i = 2010
while i <=2018:
    print("Downloading data for the", i, "season.")
    start = dt.datetime.now()
    teams = Teams(year = i)
    end = dt.datetime.now()
    teams_temp = teams.dataframes
    teams_temp['Season'] = i
    teams_df = pd.concat([teams_df, teams_temp])
    span = end - start
    print(i, "took", span, "seconds to download.")
    i += 1

2010 took 0:00:25.708131 seconds to download.
2011 took 0:00:28.452803 seconds to download.
2012 took 0:00:27.385087 seconds to download.
2013 took 0:00:30.318756 seconds to download.
2014 took 0:00:27.503782 seconds to download.
2015 took 0:00:29.890348 seconds to download.
2016 took 0:00:27.798723 seconds to download.
2017 took 0:00:27.657564 seconds to download.
2018 took 0:00:28.595215 seconds to download.


In [64]:
teams_df.columns

Index(['abbreviation', 'assist_percentage', 'assists', 'away_losses',
       'away_wins', 'block_percentage', 'blocks', 'conference',
       'conference_losses', 'conference_wins', 'defensive_rebounds',
       'effective_field_goal_percentage', 'field_goal_attempts',
       'field_goal_percentage', 'field_goals', 'free_throw_attempt_rate',
       'free_throw_attempts', 'free_throw_percentage', 'free_throws',
       'free_throws_per_field_goal_attempt', 'games_played', 'home_losses',
       'home_wins', 'losses', 'minutes_played', 'name', 'net_rating',
       'offensive_rating', 'offensive_rebound_percentage',
       'offensive_rebounds', 'opp_assist_percentage', 'opp_assists',
       'opp_block_percentage', 'opp_blocks', 'opp_defensive_rebounds',
       'opp_effective_field_goal_percentage', 'opp_field_goal_attempts',
       'opp_field_goal_percentage', 'opp_field_goals',
       'opp_free_throw_attempt_rate', 'opp_free_throw_attempts',
       'opp_free_throw_percentage', 'opp_free_thro

In [4]:
team_names = teams_df[['abbreviation']]

In [5]:
team_names.head()

,abbreviation
AIR-FORCE,AIR-FORCE
AKRON,AKRON
ALABAMA-AM,ALABAMA-AM
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM
ALABAMA-STATE,ALABAMA-STATE


In [6]:
#Only needed this on first run
#team_names.to_csv(r'data/team_names.csv')
#teams_df.to_csv(r'data/team_data.csv')

In [7]:
games = pd.read_csv(r"C:\Users\bdraus\Documents\Python Scripts\Practice\TourneyCompactResults.csv")

In [8]:
games = games[['Season','WTeamID','LTeamID']]
games['Winner'] = games['WTeamID']
games = games.rename(columns = {'WTeamID' : 'Team_A', 'LTeamID': 'Team_B'})
games.head()

,Season,Team_A,Team_B,Winner
0,1985,1116,1234,1116
1,1985,1120,1345,1120
2,1985,1207,1250,1207
3,1985,1229,1425,1229
4,1985,1242,1325,1242


In [9]:
i = 0
games['Team_X'] = 0
games['Team_Y'] = 0
while i < len(games):
    games['Team_X'][i] = random.choice(([games['Team_A'][i],games['Team_B'][i]]))
    i += 1
games['Team_Y'] = np.where(games['Team_X'] == games['Team_A'], games['Team_B'], games['Team_A'])
games['Result'] = np.where(games['Winner'] == games['Team_X'], 'X', 'Y')
games = games.drop(columns = ['Team_A','Team_B'], axis = 1)

In [10]:
games.head(25)

,Season,Winner,Team_X,Team_Y,Result
0,1985,1116,1116,1234,X
1,1985,1120,1120,1345,X
2,1985,1207,1207,1250,X
3,1985,1229,1229,1425,X
4,1985,1242,1242,1325,X
5,1985,1246,1246,1449,X
6,1985,1256,1256,1338,X
7,1985,1260,1233,1260,Y
8,1985,1314,1292,1314,Y
9,1985,1323,1323,1333,X


In [11]:
team_data = teams_df[['Season','abbreviation','assist_percentage','block_percentage','effective_field_goal_percentage','field_goal_percentage','free_throw_attempt_rate','free_throw_percentage','free_throws_per_field_goal_attempt','net_rating','offensive_rating','offensive_rebound_percentage','opp_effective_field_goal_percentage','opp_field_goal_percentage','opp_free_throw_attempt_rate','opp_free_throws_per_field_goal_attempt','opp_offensive_rating','opp_offensive_rebound_percentage','opp_steal_percentage','opp_three_point_attempt_rate','opp_three_point_field_goal_percentage','opp_total_rebound_percentage','opp_true_shooting_percentage','opp_two_point_field_goal_percentage','pace','simple_rating_system','steal_percentage','strength_of_schedule','three_point_attempt_rate','three_point_field_goal_percentage','total_rebound_percentage','true_shooting_percentage','turnover_percentage','two_point_field_goal_percentage','win_percentage']]

In [12]:
games = games.loc[games['Season'] >= 2010]
games.head()

,Season,Winner,Team_X,Team_Y,Result
1584,2010,1115,1115,1457,X
1585,2010,1124,1358,1124,Y
1586,2010,1139,1139,1431,X
1587,2010,1140,1196,1140,Y
1588,2010,1242,1242,1250,X


In [13]:
team_abb = pd.read_csv(r"C:\Users\bdraus\Documents\Python Scripts\Practice\team_names.csv")

In [14]:
team_abb = team_abb[['abbreviation','Team_Id']]

In [15]:
team_data_abb = team_data.merge(team_abb, left_on = 'abbreviation', right_on = 'abbreviation', how = 'left').dropna()
team_data_abb['Team_Id'] = team_data_abb['Team_Id'].astype(np.int64)

In [16]:
team_data_abb.head()

,Season,abbreviation,assist_percentage,block_percentage,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,free_throws_per_field_goal_attempt,net_rating,...,steal_percentage,strength_of_schedule,three_point_attempt_rate,three_point_field_goal_percentage,total_rebound_percentage,true_shooting_percentage,turnover_percentage,two_point_field_goal_percentage,win_percentage,Team_Id
0,2010,AIR-FORCE,61.6,6.2,0.504,0.443,0.367,0.635,0.233,-10.0,...,8.4,3.13,0.394,0.313,46.8,0.529,19.0,0.527,0.323,1102
1,2010,AKRON,53.9,8.5,0.491,0.433,0.363,0.657,0.239,7.6,...,9.0,-1.50,0.343,0.339,51.6,0.521,16.4,0.483,0.686,1103
2,2010,ALABAMA-AM,48.1,12.7,0.416,0.382,0.474,0.635,0.301,-5.2,...,12.5,-13.71,0.237,0.291,46.8,0.463,18.8,0.410,0.407,1105
3,2010,ALABAMA-BIRMINGHAM,51.1,7.3,0.471,0.422,0.457,0.694,0.317,10.0,...,10.0,2.90,0.315,0.311,53.6,0.518,17.0,0.474,0.735,1412
4,2010,ALABAMA-STATE,60.0,11.1,0.462,0.404,0.448,0.641,0.287,-2.1,...,10.8,-12.02,0.356,0.324,51.3,0.499,20.0,0.448,0.516,1106


In [17]:
games_a = games.merge(team_data_abb, left_on = ['Team_X','Season'], right_on = ['Team_Id','Season'], how = 'left')
games_b = games_a.merge(team_data_abb, left_on = ['Team_Y','Season'], right_on = ['Team_Id','Season'], how = 'left')
games_b = games_b.dropna()
print(games_b.head(15))

    Season  Winner  Team_X  Team_Y Result       abbreviation_x  \
0     2010    1115    1115    1457      X  ARKANSAS-PINE-BLUFF   
1     2010    1124    1358    1124      Y    SAM-HOUSTON-STATE   
2     2010    1139    1139    1431      X               BUTLER   
3     2010    1140    1196    1140      Y              FLORIDA   
4     2010    1242    1242    1250      X               KANSAS   
5     2010    1243    1317    1243      Y          NORTH-TEXAS   
6     2010    1246    1246    1190      X             KENTUCKY   
7     2010    1293    1293    1435      X         MURRAY-STATE   
8     2010    1307    1285    1307      Y              MONTANA   
9     2010    1320    1424    1320      Y     NEVADA-LAS-VEGAS   
10    2010    1325    1207    1325      Y           GEORGETOWN   
11    2010    1330    1323    1330      Y           NOTRE-DAME   
12    2010    1388    1388    1350      X       SAINT-MARYS-CA   
13    2010    1397    1397    1361      X            TENNESSEE   
14    2010

In [18]:
games_b['Winner'] = np.where(games_b['Winner'] == games_b['Team_X'], games_b['abbreviation_x'], games_b['abbreviation_y'])
games_b['Team_X'] = games_b['abbreviation_x']
games_b['Team_Y'] = games_b['abbreviation_y']

In [19]:
games_b.dtypes

Season                                        int64
Winner                                       object
Team_X                                       object
Team_Y                                       object
Result                                       object
abbreviation_x                               object
assist_percentage_x                         float64
block_percentage_x                          float64
effective_field_goal_percentage_x           float64
field_goal_percentage_x                     float64
free_throw_attempt_rate_x                   float64
free_throw_percentage_x                     float64
free_throws_per_field_goal_attempt_x        float64
net_rating_x                                float64
offensive_rating_x                          float64
offensive_rebound_percentage_x              float64
opp_effective_field_goal_percentage_x       float64
opp_field_goal_percentage_x                 float64
opp_free_throw_attempt_rate_x               float64
opp_free_thr

In [20]:
print(games_b.head(15))

    Season               Winner               Team_X                Team_Y  \
0     2010  ARKANSAS-PINE-BLUFF  ARKANSAS-PINE-BLUFF              WINTHROP   
1     2010               BAYLOR    SAM-HOUSTON-STATE                BAYLOR   
2     2010               BUTLER               BUTLER         TEXAS-EL-PASO   
3     2010        BRIGHAM-YOUNG              FLORIDA         BRIGHAM-YOUNG   
4     2010               KANSAS               KANSAS                LEHIGH   
5     2010         KANSAS-STATE          NORTH-TEXAS          KANSAS-STATE   
6     2010             KENTUCKY             KENTUCKY  EAST-TENNESSEE-STATE   
7     2010         MURRAY-STATE         MURRAY-STATE            VANDERBILT   
8     2010           NEW-MEXICO              MONTANA            NEW-MEXICO   
9     2010        NORTHERN-IOWA     NEVADA-LAS-VEGAS         NORTHERN-IOWA   
10    2010                 OHIO           GEORGETOWN                  OHIO   
11    2010         OLD-DOMINION           NOTRE-DAME          OL

In [21]:
ml_input = games_b.drop(columns = ['Season','Winner','Team_X','Team_Y','abbreviation_x','abbreviation_y','Team_Id_x','Team_Id_y','win_percentage_x','win_percentage_y'], axis = 1)
ml_input.dtypes

Result                                       object
assist_percentage_x                         float64
block_percentage_x                          float64
effective_field_goal_percentage_x           float64
field_goal_percentage_x                     float64
free_throw_attempt_rate_x                   float64
free_throw_percentage_x                     float64
free_throws_per_field_goal_attempt_x        float64
net_rating_x                                float64
offensive_rating_x                          float64
offensive_rebound_percentage_x              float64
opp_effective_field_goal_percentage_x       float64
opp_field_goal_percentage_x                 float64
opp_free_throw_attempt_rate_x               float64
opp_free_throws_per_field_goal_attempt_x    float64
opp_offensive_rating_x                      float64
opp_offensive_rebound_percentage_x          float64
opp_steal_percentage_x                      float64
opp_three_point_attempt_rate_x              float64
opp_three_po

In [22]:
X_all = ml_input.drop(['Result'],1)
y_all = ml_input['Result']

In [25]:
cols = [['assist_percentage_x','block_percentage_x','effective_field_goal_percentage_x','field_goal_percentage_x','free_throw_attempt_rate_x','free_throw_percentage_x','free_throws_per_field_goal_attempt_x','net_rating_x','offensive_rating_x','offensive_rebound_percentage_x','opp_effective_field_goal_percentage_x','opp_field_goal_percentage_x','opp_free_throw_attempt_rate_x','opp_free_throws_per_field_goal_attempt_x','opp_offensive_rating_x','opp_offensive_rebound_percentage_x','opp_steal_percentage_x','opp_three_point_attempt_rate_x','opp_three_point_field_goal_percentage_x','opp_total_rebound_percentage_x','opp_true_shooting_percentage_x','opp_two_point_field_goal_percentage_x','pace_x','simple_rating_system_x','steal_percentage_x','strength_of_schedule_x','three_point_attempt_rate_x','three_point_field_goal_percentage_x','total_rebound_percentage_x','true_shooting_percentage_x','turnover_percentage_x','two_point_field_goal_percentage_x','assist_percentage_y','block_percentage_y','effective_field_goal_percentage_y','field_goal_percentage_y','free_throw_attempt_rate_y','free_throw_percentage_y','free_throws_per_field_goal_attempt_y','net_rating_y','offensive_rating_y','offensive_rebound_percentage_y','opp_effective_field_goal_percentage_y','opp_field_goal_percentage_y','opp_free_throw_attempt_rate_y','opp_free_throws_per_field_goal_attempt_y','opp_offensive_rating_y','opp_offensive_rebound_percentage_y','opp_steal_percentage_y','opp_three_point_attempt_rate_y','opp_three_point_field_goal_percentage_y','opp_total_rebound_percentage_y','opp_true_shooting_percentage_y','opp_two_point_field_goal_percentage_y','pace_y','simple_rating_system_y','steal_percentage_y','strength_of_schedule_y','three_point_attempt_rate_y','three_point_field_goal_percentage_y','total_rebound_percentage_y','true_shooting_percentage_y','turnover_percentage_y','two_point_field_goal_percentage_y']]
for col in cols:
    X_all[col] = scale(X_all[col])

In [26]:
len(X_all)

533

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,
                                                   stratify = y_all)

In [28]:
#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    time_taken = end - start
    
    # Print the results
    print('Trained model', clf.__class__.__name__, 'in', time_taken, ' seconds')

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    time_taken = end - start
    print("Made predictions in", time_taken, "seconds.")
    
    return f1_score(target, y_pred, pos_label='X'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    #print("Training something using a training set size of %d. . .").format(len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print("F1 score and accuracy score for training set:", f1, " , ", acc)
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set:", f1, " , ", acc)

In [29]:
clf_A = LogisticRegression(random_state = 64)
clf_B = SVC(random_state = 64, kernel = 'rbf')
clf_C = xgb.XGBClassifier(seed = 64)

train_predict(clf_A, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_B, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_C, X_train, y_train, X_test, y_test)
print('')

Trained model LogisticRegression in 0.01804804801940918  seconds
Made predictions in 0.0010035037994384766 seconds.
F1 score and accuracy score for training set: 0.7357512953367875  ,  0.7443609022556391
Made predictions in 0.0010008811950683594 seconds.
F1 score and accuracy score for test set: 0.7313432835820896  ,  0.7313432835820896

Trained model SVC in 0.019019365310668945  seconds
Made predictions in 0.015039205551147461 seconds.
F1 score and accuracy score for training set: 0.8906666666666666  ,  0.8972431077694235
Made predictions in 0.00902414321899414 seconds.
F1 score and accuracy score for test set: 0.6504065040650406  ,  0.6791044776119403

Trained model XGBClassifier in 0.27976512908935547  seconds
Made predictions in 0.009022235870361328 seconds.
F1 score and accuracy score for training set: 0.9974025974025974  ,  0.9974937343358395
Made predictions in 0.0030107498168945312 seconds.
F1 score and accuracy score for test set: 0.703125  ,  0.7164179104477612



C:\Users\bdraus\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\bdraus\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [30]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer


# TODO: Create the parameters list you wish to tune
parameters = { 'learning_rate' : [0.1],
               'n_estimators' : [40],
               'max_depth': [3],
               'min_child_weight': [3],
               'gamma':[0.4],
               'subsample' : [0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             }  

# TODO: Initialize the classifier
clf = xgb.XGBClassifier(seed=2)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='X')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
f1, acc = predict_labels(clf, X_train, y_train)
print("F1 score and accuracy score for training set:", f1, " , ", acc)
    
f1, acc = predict_labels(clf, X_test, y_test)
print("F1 score and accuracy score for test set:", f1, " , ", acc)

C:\Users\bdraus\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\bdraus\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\bdraus\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in a

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
       n_estimators=40, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=1e-05,
       reg_lambda=1, scale_pos_weight=1, seed=2, silent=True,
       subsample=0.8)
Made predictions in 0.0050144195556640625 seconds.
F1 score and accuracy score for training set: 0.9124668435013263  ,  0.9172932330827067
Made predictions in 0.0030083656311035156 seconds.
F1 score and accuracy score for test set: 0.71875  ,  0.7313432835820896


C:\Users\bdraus\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
X_all.head()

,assist_percentage_x,block_percentage_x,effective_field_goal_percentage_x,field_goal_percentage_x,free_throw_attempt_rate_x,free_throw_percentage_x,free_throws_per_field_goal_attempt_x,net_rating_x,offensive_rating_x,offensive_rebound_percentage_x,...,pace_y,simple_rating_system_y,steal_percentage_y,strength_of_schedule_y,three_point_attempt_rate_y,three_point_field_goal_percentage_y,total_rebound_percentage_y,true_shooting_percentage_y,turnover_percentage_y,two_point_field_goal_percentage_y
0,0.608412,-0.235251,-2.674008,-2.309648,2.110936,-1.123586,1.545718,-2.297406,-3.368679,0.422575,...,-0.031237,-2.887607,1.558432,-2.586309,-1.589965,-4.130417,-0.300089,-4.096505,-0.037814,-2.823171
1,3.869504,-1.308955,0.614823,0.161513,-0.232312,-0.134183,-0.275719,0.107664,0.306993,0.251667,...,0.127026,0.524071,-0.173357,0.502712,-0.221704,0.893355,1.071466,1.027070,1.161876,0.996697
2,-0.047908,-1.476722,-0.345959,-0.782021,1.802614,0.855219,2.210687,0.155766,-0.631901,-0.847028,...,0.949997,-0.155704,1.334975,-0.719606,-0.491210,-0.646779,-0.501789,0.073846,0.682000,0.927245
3,-0.273518,-1.241849,-0.937210,-0.692161,-0.972285,-0.134183,-0.998511,-0.774195,-0.512042,0.764391,...,1.329829,0.632887,1.055654,-0.510181,-0.118047,2.066791,0.506708,1.662552,-0.577674,0.302175
4,0.793002,1.710839,1.169120,1.329698,0.651545,-0.247258,0.533809,1.839314,1.365746,1.106207,...,1.013302,-2.202711,-0.731999,-2.682967,-0.802178,1.406733,-0.340429,0.034129,0.562031,-1.017415


This section inputs all upsets from 2010-2018 and tests how the model performs

In [32]:
game_list = [[2017,'MIDDLE-TENNESSEE','MINNESOTA','MIDDLE-TENNESSEE'],
[2016,'ARKANSAS-LITTLE-ROCK','PURDUE','ARKANSAS-LITTLE-ROCK'],
[2016,'YALE','BAYLOR','YALE'],
[2014,'STEPHEN-F-AUSTIN','VIRGINIA-COMMONWEALTH','STEPHEN-F-AUSTIN'],
[2014,'NORTH-DAKOTA-STATE','OKLAHOMA','NORTH-DAKOTA-STATE'],
[2014,'HARVARD','CINCINNATI','HARVARD'],
[2013,'MISSISSIPPI','WISCONSIN','MISSISSIPPI'],
[2013,'CALIFORNIA','NEVADA-LAS-VEGAS','CALIFORNIA'],
[2013,'OREGON','OKLAHOMA-STATE','OREGON'],
[2012,'SOUTH-FLORIDA','TEMPLE','SOUTH-FLORIDA'],
[2012,'VIRGINIA-COMMONWEALTH','WICHITA-STATE','VIRGINIA-COMMONWEALTH'],
[2011,'RICHMOND','VANDERBILT','RICHMOND'],
[2010,'CORNELL','TEMPLE','CORNELL'],
[2016,'NORTHERN-IOWA','TEXAS','NORTHERN-IOWA'],
[2016,'GONZAGA','SETON-HALL','GONZAGA'],
[2016,'WICHITA-STATE','ARIZONA','WICHITA-STATE'],
[2015,'DAYTON','PROVIDENCE','DAYTON'],
[2015,'UCLA','SOUTHERN-METHODIST','UCLA'],
[2014,'TENNESSEE','MASSACHUSETTS','TENNESSEE'],
[2014,'DAYTON','OHIO-STATE','DAYTON'],
[2013,'MINNESOTA','UCLA','MINNESOTA'],
[2016,'MIDDLE-TENNESSEE','MICHIGAN-STATE','MIDDLE-TENNESSEE'],
[2013,'FLORIDA-GULF-COAST','GEORGETOWN','FLORIDA-GULF-COAST'],
[2012,'LEHIGH','DUKE','LEHIGH'],
[2012,'NORFOLK-STATE','MISSOURI','NORFOLK-STATE'],
[2016,'STEPHEN-F-AUSTIN','WEST-VIRGINIA','STEPHEN-F-AUSTIN'],
[2015,'GEORGIA-STATE','BAYLOR','GEORGIA-STATE'],
[2015,'ALABAMA-BIRMINGHAM','IOWA-STATE','ALABAMA-BIRMINGHAM'],
[2014,'MERCER','DUKE','MERCER'],
[2013,'HARVARD','NEW-MEXICO','HARVARD'],
[2010,'OHIO','GEORGETOWN','OHIO'],
[2018,'MARSHALL','WICHITA-STATE','MARSHALL'],
[2018,'BUFFALO','ARIZONA','BUFFALO'],
[2016,'HAWAII','CALIFORNIA','HAWAII'],
[2013,'LA-SALLE','KANSAS-STATE','LA-SALLE'],
[2012,'OHIO','MICHIGAN','OHIO'],
[2011,'MOREHEAD-STATE','LOUISVILLE','MOREHEAD-STATE'],
[2010,'MURRAY-STATE','VANDERBILT','MURRAY-STATE'],
[2018,'LOYOLA-IL','MIAMI-FL','LOYOLA-IL'],
[2018,'SYRACUSE','TEXAS-CHRISTIAN','SYRACUSE'],
[2017,'RHODE-ISLAND','CREIGHTON','RHODE-ISLAND'],
[2017,'SOUTHERN-CALIFORNIA','SOUTHERN-METHODIST','SOUTHERN-CALIFORNIA'],
[2017,'XAVIER','MARYLAND','XAVIER']]
game_df_teams = pd.DataFrame(game_list)
game_df_teams.columns = ['Season','Away','Home','Actual Winner']
game_df_teams

,Season,Away,Home,Actual Winner
0,2017,MIDDLE-TENNESSEE,MINNESOTA,MIDDLE-TENNESSEE
1,2016,ARKANSAS-LITTLE-ROCK,PURDUE,ARKANSAS-LITTLE-ROCK
2,2016,YALE,BAYLOR,YALE
3,2014,STEPHEN-F-AUSTIN,VIRGINIA-COMMONWEALTH,STEPHEN-F-AUSTIN
4,2014,NORTH-DAKOTA-STATE,OKLAHOMA,NORTH-DAKOTA-STATE
5,2014,HARVARD,CINCINNATI,HARVARD
6,2013,MISSISSIPPI,WISCONSIN,MISSISSIPPI
7,2013,CALIFORNIA,NEVADA-LAS-VEGAS,CALIFORNIA
8,2013,OREGON,OKLAHOMA-STATE,OREGON
9,2012,SOUTH-FLORIDA,TEMPLE,SOUTH-FLORIDA


In [33]:
game_df = pd.DataFrame(game_list)
game_df.columns = ['Season','Away','Home','Actual Winner']
game_df = game_df[['Season','Away','Home']]
game_df = game_df.merge(team_data_abb, left_on = ['Season','Home'], right_on = ['Season','abbreviation'], how = 'left')
game_df = game_df.merge(team_data_abb, left_on = ['Season','Away'], right_on = ['Season','abbreviation'], how = 'left')
print(game_df)

    Season                   Away                   Home  \
0     2017       MIDDLE-TENNESSEE              MINNESOTA   
1     2016   ARKANSAS-LITTLE-ROCK                 PURDUE   
2     2016                   YALE                 BAYLOR   
3     2014       STEPHEN-F-AUSTIN  VIRGINIA-COMMONWEALTH   
4     2014     NORTH-DAKOTA-STATE               OKLAHOMA   
5     2014                HARVARD             CINCINNATI   
6     2013            MISSISSIPPI              WISCONSIN   
7     2013             CALIFORNIA       NEVADA-LAS-VEGAS   
8     2013                 OREGON         OKLAHOMA-STATE   
9     2012          SOUTH-FLORIDA                 TEMPLE   
10    2012  VIRGINIA-COMMONWEALTH          WICHITA-STATE   
11    2011               RICHMOND             VANDERBILT   
12    2010                CORNELL                 TEMPLE   
13    2016          NORTHERN-IOWA                  TEXAS   
14    2016                GONZAGA             SETON-HALL   
15    2016          WICHITA-STATE       

In [34]:
game_input = game_df.drop(columns = ['Season','Away','Home','abbreviation_x','abbreviation_y','Team_Id_x','Team_Id_y','win_percentage_x','win_percentage_y'], axis = 1)

In [35]:
game_input

,assist_percentage_x,block_percentage_x,effective_field_goal_percentage_x,field_goal_percentage_x,free_throw_attempt_rate_x,free_throw_percentage_x,free_throws_per_field_goal_attempt_x,net_rating_x,offensive_rating_x,offensive_rebound_percentage_x,...,pace_y,simple_rating_system_y,steal_percentage_y,strength_of_schedule_y,three_point_attempt_rate_y,three_point_field_goal_percentage_y,total_rebound_percentage_y,true_shooting_percentage_y,turnover_percentage_y,two_point_field_goal_percentage_y
0,57.2,15.9,0.487,0.435,0.396,0.711,0.281,8.0,104.6,29.6,...,66.4,9.80,10.1,-0.48,0.299,0.367,53.9,0.567,13.6,0.540
1,64.1,11.7,0.536,0.470,0.354,0.744,0.263,18.5,113.5,34.5,...,64.6,7.12,10.1,-1.76,0.354,0.384,50.1,0.553,13.6,0.490
2,64.6,11.2,0.518,0.466,0.375,0.725,0.272,10.9,112.8,40.1,...,66.9,9.08,8.4,-1.03,0.309,0.363,57.8,0.555,16.4,0.517
3,49.8,11.5,0.479,0.420,0.352,0.674,0.237,13.7,105.0,34.3,...,65.5,4.82,11.1,-6.76,0.360,0.345,53.8,0.561,14.2,0.529
4,53.1,8.4,0.517,0.447,0.394,0.749,0.295,8.1,112.9,32.4,...,63.9,6.31,9.7,-0.02,0.275,0.364,53.2,0.593,13.4,0.558
5,52.8,16.0,0.477,0.425,0.387,0.708,0.274,15.6,107.5,39.1,...,64.6,10.79,11.6,-0.47,0.283,0.385,52.1,0.564,15.0,0.496
6,56.5,10.8,0.486,0.420,0.297,0.634,0.188,14.5,104.4,32.4,...,70.5,14.42,11.6,4.14,0.330,0.324,48.1,0.526,13.0,0.494
7,64.4,14.6,0.492,0.436,0.373,0.703,0.262,12.1,103.2,33.8,...,66.5,10.65,8.7,7.77,0.236,0.302,50.0,0.515,16.2,0.488
8,48.9,12.8,0.488,0.439,0.405,0.746,0.302,12.9,104.8,31.3,...,68.8,13.82,12.1,5.77,0.273,0.333,52.5,0.531,18.2,0.491
9,57.3,10.1,0.531,0.469,0.342,0.719,0.246,7.9,108.8,31.5,...,59.7,9.20,9.2,7.29,0.303,0.316,50.5,0.521,19.1,0.490


In [36]:
y_pred_C = clf_C.predict(game_input)
y_pred_B = clf_B.predict(game_input)
y_pred_A = clf_A.predict(game_input)
Y_pred = clf.predict(game_input)

C:\Users\bdraus\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\bdraus\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [37]:
print(y_pred_C,y_pred_B,y_pred_A,Y_pred)

['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'X' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'] ['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'] ['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'] ['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'X' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y']


In [38]:
test = pd.DataFrame(Y_pred)

In [39]:
games_df_out = game_df_teams.merge(test, left_index = True, right_index = True)
games_df_out.columns = ['Season','Away','Home','Actual Winner','Forecast']

In [40]:
games_df_out['Winner'] = np.where(games_df_out['Forecast'] == 'Y', games_df_out['Away'], games_df_out['Home'])
games_df_out['Correct'] = np.where(games_df_out['Actual Winner'] == games_df_out['Winner'], "Correct!", "¯\_(ツ)_/¯")

In [41]:
games_df_out[['Season','Away','Home','Actual Winner','Winner','Correct']]

,Season,Away,Home,Actual Winner,Winner,Correct
0,2017,MIDDLE-TENNESSEE,MINNESOTA,MIDDLE-TENNESSEE,MIDDLE-TENNESSEE,Correct!
1,2016,ARKANSAS-LITTLE-ROCK,PURDUE,ARKANSAS-LITTLE-ROCK,ARKANSAS-LITTLE-ROCK,Correct!
2,2016,YALE,BAYLOR,YALE,YALE,Correct!
3,2014,STEPHEN-F-AUSTIN,VIRGINIA-COMMONWEALTH,STEPHEN-F-AUSTIN,STEPHEN-F-AUSTIN,Correct!
4,2014,NORTH-DAKOTA-STATE,OKLAHOMA,NORTH-DAKOTA-STATE,NORTH-DAKOTA-STATE,Correct!
5,2014,HARVARD,CINCINNATI,HARVARD,HARVARD,Correct!
6,2013,MISSISSIPPI,WISCONSIN,MISSISSIPPI,MISSISSIPPI,Correct!
7,2013,CALIFORNIA,NEVADA-LAS-VEGAS,CALIFORNIA,CALIFORNIA,Correct!
8,2013,OREGON,OKLAHOMA-STATE,OREGON,OREGON,Correct!
9,2012,SOUTH-FLORIDA,TEMPLE,SOUTH-FLORIDA,SOUTH-FLORIDA,Correct!


This section is where you can input current year games and get an output.

In [53]:
teams_19 = Teams(year = 2019)
teams_df_19 = teams.dataframes
teams_df_19 = teams_df_19[['abbreviation','assist_percentage','block_percentage','effective_field_goal_percentage','field_goal_percentage','free_throw_attempt_rate','free_throw_percentage','free_throws_per_field_goal_attempt','net_rating','offensive_rating','offensive_rebound_percentage','opp_effective_field_goal_percentage','opp_field_goal_percentage','opp_free_throw_attempt_rate','opp_free_throws_per_field_goal_attempt','opp_offensive_rating','opp_offensive_rebound_percentage','opp_steal_percentage','opp_three_point_attempt_rate','opp_three_point_field_goal_percentage','opp_total_rebound_percentage','opp_true_shooting_percentage','opp_two_point_field_goal_percentage','pace','simple_rating_system','steal_percentage','strength_of_schedule','three_point_attempt_rate','three_point_field_goal_percentage','total_rebound_percentage','true_shooting_percentage','turnover_percentage','two_point_field_goal_percentage']]

In [54]:
teams_df_19.head()

,abbreviation,assist_percentage,block_percentage,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,free_throws_per_field_goal_attempt,net_rating,offensive_rating,...,pace,simple_rating_system,steal_percentage,strength_of_schedule,three_point_attempt_rate,three_point_field_goal_percentage,total_rebound_percentage,true_shooting_percentage,turnover_percentage,two_point_field_goal_percentage
ABILENE-CHRISTIAN,ABILENE-CHRISTIAN,55.4,11.6,0.521,0.464,0.309,0.701,0.217,3.5,102.2,...,71.6,-9.14,11.3,-6.82,0.350,0.325,49.7,0.549,17.7,0.539
AIR-FORCE,AIR-FORCE,60.7,8.1,0.490,0.419,0.318,0.734,0.233,-5.7,100.8,...,67.7,-4.31,9.5,1.72,0.431,0.331,48.7,0.527,16.3,0.485
AKRON,AKRON,52.7,7.5,0.518,0.435,0.319,0.696,0.222,-5.1,102.6,...,69.1,-6.82,8.4,-1.92,0.467,0.358,49.2,0.547,17.3,0.502
ALABAMA-AM,ALABAMA-AM,50.5,3.9,0.450,0.397,0.314,0.647,0.203,-23.2,88.1,...,68.3,-23.97,5.8,-8.04,0.354,0.303,48.2,0.480,20.9,0.448
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,59.3,11.6,0.545,0.488,0.291,0.750,0.218,10.1,109.8,...,69.5,4.90,7.7,-0.65,0.334,0.345,54.8,0.575,16.5,0.559


In [55]:
test = teams_df_19[teams_df_19['abbreviation']=='CENTRAL-FLORIDA']

In [56]:
test

,abbreviation,assist_percentage,block_percentage,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,free_throws_per_field_goal_attempt,net_rating,offensive_rating,...,pace,simple_rating_system,steal_percentage,strength_of_schedule,three_point_attempt_rate,three_point_field_goal_percentage,total_rebound_percentage,true_shooting_percentage,turnover_percentage,two_point_field_goal_percentage
CENTRAL-FLORIDA,CENTRAL-FLORIDA,49.1,9.4,0.478,0.424,0.431,0.641,0.277,1.5,96.2,...,64.9,4.75,8.1,3.75,0.331,0.329,50.5,0.512,18.0,0.471


In [57]:
first_round = [['NORTH_CAROLINA-CENTRAL','NORTH-DAKOTA-STATE'],['NORTH_DAKOTA-STATE','DUKE'],
               ['VIRGINIA-COMMONWEALTH','CENTRAL-FLORIDA'],['MISSISSIPPI_STATE','LIBERTY'],['VIRGINIA_TECH','SAINT-LOUIS'],
               ['BELMONT','TEMPLE'],['MARYLAND','TEMPLE'] ,['LOUISIANA-STATE','YALE'],['LOUISVILLE','MINNESOTA'],
               ['BRADLEY','MICHIGAN-STATE'],['FAIRLEIGH-DICKINSON','PRAIRIE-VIEW'],['PRAIRIE-VIEW','GONZAGA'],
               ['SYRACUSE','BAYLOR'],['MARQUETTE','MURRAY-STATE'],['FLORIDA-STATE','VERMONT'],['ST-JOHNS-NY','ARIZONA-STATE'],
               ['ST-JOHNS-NY','BUFFALO'],['TEXAS-TECH','NORTHERN-KENTUCKY'],['FLORIDA','NEVADA'],['MICHIGAN','MONTANA'],
               ['GARDNER-WEBB','VIRGINIA'],['MISSISSIPPI','OKLAHOMA'],['WISCONSIN','OREGON'], 
               ['KANSAS-STATE','CALIFORNIA-IRVINE'],['VILLANOVA','SAINT-MARYS-CA'], ['PURDUE','OLD-DOMINION'],
               ['CINCINNNATI','IOWA'],['TENNESSEE','COLGATE'],['NORTH-CAROLINA','IONA'],['UTAH-STATE','WASHINGTON'],
               ['AUBURN','NEW-MEXICO-STATE'],['KANSAS','NORTHEASTERN'],['IOWA-STATE','OHIO-STATE'],['HOUSTON','GEORGIA-STATE'],
               ['WOFFORD','SETON-HALL'],['KENTUCKY','ABILENE-CHRISTIAN']]

second_round = [['DUKE','VIRGINIA-COMMONWEALTH'],['LIBERTY','SAINT-LOUIS'],['LOUISIANA-STATE','MARYLAND'],
                  ['LOUISVILLE','BRADLEY'],['GONZAGA','SYRACUSE'],['MARQUETTE','FLORIDA-STATE'],['ST-JOHNS-NY','TEXAS-TECH']
                 ,['FLORIDA','MICHIGAN'],['VIRGINIA','MISSISSIPPI'],['WISCONSIN','KANSAS-STATE'],['VILLANOVA','PURDUE'],
                 ['IOWA','TENNESSEE'],['NORTH-CAROLINA','UTAH-STATE'],['AUBURN','KANSAS'],['IOWA-STATE','HOUSTON'],
                 ['SETON-HALL','KENTUCKY']]

sweet_sixteen = [['SAINT-LOUIS','VIRGINIA-COMMONWEALTH'],['MARYLAND','LOUISVILLE'],['SYRACUSE','FLORIDA-STATE'],
                  ['ST-JOHNS-NY','FLORIDA'],['VIRGINIA','WISCONSIN'],['TENNESSEE','PURDUE'],['NORTH-CAROLINA','AUBURN']
                 ,['IOWA-STATE','SETON-HALL']]

elite_eight = [['LOUISVILLE','VIRGINIA-COMMONWEALTH'],['FLORIDA-STATE','ST-JOHNS-NY'],['WISCONSIN','PURDUE'],
                  ['NORTH-CAROLINA','IOWA-STATE']]

final_four = [['ST-JOHNS-NY','VIRGINIA-COMMONWEALTH'],['PURDUE','NORTH-CAROLINA']]

finals = [['NORTH-CAROLINA','VIRGINIA-COMMONWEALTH']]


game_df_cur_team = pd.DataFrame(first_round)
game_df_cur_team.columns = ['Away','Home']

In [58]:
game_df_cur = pd.DataFrame(first_round)
game_df_cur.columns = ['Away','Home']
game_df_cur = game_df_cur[['Away','Home']]
game_df_cur = game_df_cur.merge(teams_df_19, left_on = ['Home'], right_on = ['abbreviation'], how = 'left')
game_df_cur = game_df_cur.merge(teams_df_19, left_on = ['Away'], right_on = ['abbreviation'], how = 'left')

In [59]:
game_df_cur.columns

Index(['Away', 'Home', 'abbreviation_x', 'assist_percentage_x',
       'block_percentage_x', 'effective_field_goal_percentage_x',
       'field_goal_percentage_x', 'free_throw_attempt_rate_x',
       'free_throw_percentage_x', 'free_throws_per_field_goal_attempt_x',
       'net_rating_x', 'offensive_rating_x', 'offensive_rebound_percentage_x',
       'opp_effective_field_goal_percentage_x', 'opp_field_goal_percentage_x',
       'opp_free_throw_attempt_rate_x',
       'opp_free_throws_per_field_goal_attempt_x', 'opp_offensive_rating_x',
       'opp_offensive_rebound_percentage_x', 'opp_steal_percentage_x',
       'opp_three_point_attempt_rate_x',
       'opp_three_point_field_goal_percentage_x',
       'opp_total_rebound_percentage_x', 'opp_true_shooting_percentage_x',
       'opp_two_point_field_goal_percentage_x', 'pace_x',
       'simple_rating_system_x', 'steal_percentage_x',
       'strength_of_schedule_x', 'three_point_attempt_rate_x',
       'three_point_field_goal_percentage_x',

In [60]:
game_input_cur = game_df_cur.drop(columns = ['Away','Home','abbreviation_x','abbreviation_y'], axis = 1)

In [61]:
out_19 = clf.predict(game_input_cur)

C:\Users\bdraus\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [62]:
temp = pd.DataFrame(out_19)
games_df_out_cur = game_df_cur_team.merge(temp, left_index = True, right_index = True)
games_df_out_cur.columns = ['Away','Home','Forecast']
games_df_out_cur['Winner'] = np.where(games_df_out_cur['Forecast'] == 'Y', games_df_out_cur['Away'], games_df_out_cur['Home'])

In [63]:
games_df_out_cur

,Away,Home,Forecast,Winner
0,NORTH_CAROLINA-CENTRAL,NORTH-DAKOTA-STATE,Y,NORTH_CAROLINA-CENTRAL
1,NORTH_DAKOTA-STATE,DUKE,X,DUKE
2,VIRGINIA-COMMONWEALTH,CENTRAL-FLORIDA,Y,VIRGINIA-COMMONWEALTH
3,MISSISSIPPI_STATE,LIBERTY,Y,MISSISSIPPI_STATE
4,VIRGINIA_TECH,SAINT-LOUIS,X,SAINT-LOUIS
5,BELMONT,TEMPLE,Y,BELMONT
6,MARYLAND,TEMPLE,Y,MARYLAND
7,LOUISIANA-STATE,YALE,Y,LOUISIANA-STATE
8,LOUISVILLE,MINNESOTA,Y,LOUISVILLE
9,BRADLEY,MICHIGAN-STATE,Y,BRADLEY
